# Julia Test Notebook

This notebook is to test all necessary notebooks in Julia 1.9

In [1]:
using CUDA
using Flux
using Flux: Chain, Dense, gpu, cpu, params, sigmoid, train!
using BSON: @save, @load

CUDA.functional() ? println("CUDA is working") : println("CUDA is not working")

CUDA.versioninfo()
CUDA.reclaim()
CUDA.memory_status()

CUDA is working


CUDA runtime 12.1, artifact installation
CUDA driver 12.1
NVIDIA driver 531.68.0

Libraries: 
- CUBLAS: 12.1.3


- CURAND: 10.3.2
- CUFFT: 11.0.2
- CUSOLVER: 11.4.5


- CUSPARSE: 12.1.0
- CUPTI: 18.0.0
- NVML: 12.0.0+531.68

Toolchain:
- Julia: 1.9.0-rc2
- LLVM: 14.0.6
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5
- Device capability support: sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

1 device:
  0: NVIDIA GeForce RTX 3070 Laptop GPU (sm_86, 5.626 GiB / 8.000 GiB available)


Effective GPU memory usage: 12.95% (1.036 GiB/8.000 GiB)
Memory pool usage: 0 bytes (0 bytes reserved)


In [2]:
# Define a simple model
model = Chain(
    Dense(2 => 3, sigmoid),
    Dense(3 => 1)
)

model = model |> gpu

# Define a loss function
loss(x, y) = Flux.mse(model(x), y)

println("model setup finished")

model setup finished


In [3]:
# Define some training data
x = rand(Float32,2, 100) |> gpu
y = model(x)

# Train the model
Flux.train!(loss, params(model), [(x, y)], ADAM())

println("model training finished")

model training finished


In [4]:
# Save the model
model = cpu(model)
@save "model.bson" model
model = nothing

println("model saved")

model saved


In [5]:
# Load the model
@load "model.bson" model

println("model loaded")

model loaded


In [6]:
# Test the model on cpu and gpu
x = rand(Float32,2, 100)
y = model(x)

x = rand(Float32,2, 100) |> gpu
model = gpu(model)
y = model(x)

println("test done")

test done
